In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [4]:

def plot_patch_with_pca_plane(patch_path, plane_basis_path):
    # Load patch and PCA plane basis
    vol = np.load(patch_path)  # shape (D, H, W)
    plane_basis = np.load(plane_basis_path)  # shape (2, 3)

    # Compute patch center as origin for the plane
    center = np.array(vol.shape) / 2  # (z, y, x)
    origin = center[::-1]  # convert to (x, y, z)

    # Get coordinates for plane vectors
    vec1, vec2 = plane_basis  # both are (3,)

    # Create a meshgrid in the plane coordinate system
    grid_range = np.linspace(-8, 8, 20)  # Adjust for patch size
    u, v = np.meshgrid(grid_range, grid_range)
    points = origin.reshape(1, 1, 3) + u[..., None]*vec1 + v[..., None]*vec2
    X, Y, Z = points[..., 0], points[..., 1], points[..., 2]

    # Threshold the volume
    zz, yy, xx = np.where(vol > vol.mean())  # you can tune the threshold

    # Plot
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(xx, yy, zz, c=vol[zz, yy, xx], cmap='gray', alpha=0.3, s=1)
    ax.plot_surface(X, Y, Z, color='red', alpha=0.4)

    ax.set_xlim(0, vol.shape[2])
    ax.set_ylim(0, vol.shape[1])
    ax.set_zlim(0, vol.shape[0])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.title(patch_path.split("/")[-1])
    plt.tight_layout()
    plt.show()

id = "0099"
results_path="/data1/user/Umamba_data/nnUNet_results/Dataset433_ChoroidPlexus_T1xFLAIR_sym_UMAMBA/"
trainer_folder ="nnUNetTrainerMambaFirstStem_PCA_32__nnUNetPlans_First_PCA_32__3d_fullres/"
fold = 0

patch_path=results_path + trainer_folder + "fold_" + str(fold) + "/patch_"+id+".npy"
plane_basis_path=results_path + trainer_folder + "fold_" + str(fold) + "/pca_plane_"+id+".npy"
plot_patch_with_pca_plane(
    patch_path=patch_path,
    plane_basis_path=plane_basis_path
)


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from skimage.measure import marching_cubes
import nibabel as nib

def plot_patch_isosurface_with_pca_plane(patch_path, plane_basis_path, iso_threshold=None):
    # Load volume and PCA basis
    vol = np.load(patch_path)  # shape: (D, H, W)
    plane_basis = np.load(plane_basis_path)  # shape: (2, 3)

    # Determine threshold for isosurface
    if iso_threshold is None:
        iso_threshold = vol.mean()

    # Compute isosurface
    verts, faces, normals, values = marching_cubes(vol, level=iso_threshold)

    # Compute patch center
    center = np.array(vol.shape) / 2  # (z, y, x)
    origin = center[::-1]  # as (x, y, z)

    # Plane spanning vectors
    vec1, vec2 = plane_basis

    # Generate grid in plane space
    grid_range = np.linspace(-2, 2, 20)
    u, v = np.meshgrid(grid_range, grid_range)
    points = origin.reshape(1, 1, 3) + u[..., None] * vec1 + v[..., None] * vec2
    X, Y, Z = points[..., 0], points[..., 1], points[..., 2]

    # Plot
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111, projection='3d')

    # Plot isosurface
    ax.plot_trisurf(verts[:, 0], verts[:, 1], faces, verts[:, 2],
                    cmap='Spectral', lw=0.5, alpha=0.6)

    # Plot PCA plane
    ax.plot_surface(X, Y, Z, color='red', alpha=0.4)
    # Plot arrows for plane vectors
    arrow_length = 2  
    ax.quiver(
        origin[0], origin[1], origin[2],
        vec1[0], vec1[1], vec1[2],
        length=arrow_length, color='blue', linewidth=2, arrow_length_ratio=0.2
    )
    ax.quiver(
        origin[0], origin[1], origin[2],
        vec2[0], vec2[1], vec2[2],
        length=arrow_length, color='blue', linewidth=2, arrow_length_ratio=0.2
    )

    # --- Save plane as NIfTI ---

    # Create a binary mask for the plane in an 8x8x8 volume
    vol_shape = (8, 8, 8)
    plane_mask = np.zeros(vol_shape, dtype=np.uint8)

    # Generate dense grid of coordinates in the 8x8x8 volume
    zz, yy, xx = np.meshgrid(
        np.arange(vol_shape[0]), np.arange(vol_shape[1]), np.arange(vol_shape[2]), indexing='ij'
    )
    coords = np.stack([xx, yy, zz], axis=-1).reshape(-1, 3)

    # Center of the 8x8x8 volume
    center_888 = np.array(vol_shape) / 2

    # Plane equation: (p - center) dot n = 0, where n = vec1 x vec2
    n = np.cross(vec1, vec2)
    n = n / np.linalg.norm(n)
    dists = np.dot(coords - center_888[::-1], n)
    mask = np.abs(dists) < 0.5  # within half voxel

    # Set mask
    plane_mask.reshape(-1)[mask] = 1

    # Save as NIfTI
    affine = np.eye(4)
    nifti_img = nib.Nifti1Image(plane_mask, affine)
    nib.save(nifti_img, "pca_plane_8x8x8_99.nii.gz")

    ax.set_xlim(0, vol.shape[2])
    ax.set_xlim(0, vol.shape[2])
    ax.set_ylim(0, vol.shape[1])
    ax.set_zlim(0, vol.shape[0])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(patch_path.split('/')[-1])
    plt.tight_layout()
    plt.show()

id = "0099"
results_path="/data1/user/Umamba_data/nnUNet_results/Dataset433_ChoroidPlexus_T1xFLAIR_sym_UMAMBA/"
trainer_folder ="nnUNetTrainerMambaFirstStem_PCA_32__nnUNetPlans_First_PCA_32__3d_fullres/"
fold = 0
patch_path=results_path + trainer_folder + "fold_" + str(fold) + "/patch_"+id+".npy"
plane_basis_path=results_path + trainer_folder + "fold_" + str(fold) + "/pca_plane_"+id+".npy"


plot_patch_isosurface_with_pca_plane(
    patch_path=patch_path,
    plane_basis_path=plane_basis_path
)


In [18]:
import numpy as np
import pyvista as pv

def plot_patch_volume_with_pca_plane(patch_path, plane_basis_path):
    # Load data
    vol = np.load(patch_path)
    plane_basis = np.load(plane_basis_path)

    # Convert to pyvista UniformGrid
    grid = pv.UniformGrid()
    grid.dimensions = np.array(vol.shape)[::-1] + 1  # +1 to include last voxel face
    grid.origin = (0, 0, 0)
    grid.spacing = (1, 1, 1)
    grid.cell_data["values"] = vol.flatten(order="F")  # Fortran order!

    # Create a PCA plane as a rectangular surface
    center = np.array(vol.shape) / 2  # (z, y, x)
    origin = center[::-1]  # as (x, y, z)
    vec1, vec2 = plane_basis

    plane_size = 5  # side length
    grid_range = np.linspace(-plane_size/2, plane_size/2, 2)
    u, v = np.meshgrid(grid_range, grid_range)
    plane_pts = origin + u[..., None]*vec1 + v[..., None]*vec2
    plane_pts = plane_pts.reshape(-1, 3)

    # Create a surface mesh (2x2 quad)
    faces = [4, 0, 1, 3, 2]  # quad face with 4 vertices
    plane_mesh = pv.PolyData(plane_pts, faces)

    # Plot
    p = pv.Plotter()
    p.add_volume(grid, opacity="linear", cmap="gray", shade=False)
    p.add_mesh(plane_mesh, color="red", opacity=0.5)
    p.show_grid()
    p.show()

# Example usage
plot_patch_volume_with_pca_plane(
    patch_path=patch_path,
    plane_basis_path=plane_basis_path
)

AttributeError: module 'pyvista' has no attribute 'UniformGrid'

In [ ]:
import glob
patch_paths = sorted(glob.glob("/data1/user/.../patch_*.npy"))
plane_paths = sorted(glob.glob("/data1/user/.../pca_plane_*.npy"))

for patch, plane in zip(patch_paths, plane_paths):
    plot_patch_with_pca_plane(patch, plane)
